In [ ]:
import pandas as pd
import re

# Load the CSV file
data = pd.read_csv("../../../datasets/train.csv")

# Filter to only include item_weight entity
item_weight_data = data[data['entity_name'] == 'height']

# Extract numeric value and unit from entity_value
def split_value(value):
    value = value.strip()
    parts = value.strip().split(' ')
    if len(parts) == 2:
        try: 
            float(parts[0])
        except:
            return None, None
        return float(parts[0]), parts[1]
    return None, None

item_weight_data[['numeric_value', 'unit']] = item_weight_data['entity_value'].apply(split_value).apply(pd.Series)

# Drop rows with missing values
item_weight_data.dropna(inplace=True)

# Save processed data to a new CSV file
item_weight_data.to_csv('updates.csv', index=True)

print("Data preprocessing complete. Processed data saved to 'processed_item_weight_data.csv'.")

In [2]:
from utils_height import download_images
download_images(item_weight_data['image_link'].tolist(),  item_weight_data.index.tolist(), "dataset")

100%|██████████| 43597/43597 [00:01<00:00, 32549.49it/s]


In [7]:
import os
import logging
import pandas as pd
from paddleocr import PaddleOCR
import easyocr
from tqdm import tqdm

# logging.getLogger('ppocr').setLevel(logging.ERROR)
logging.getLogger('easyocr').setLevel(logging.ERROR)

image_folder = 'dataset/'  # Adjust this to the correct folder containing your images
# ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Initialize PaddleOCR
reader = easyocr.Reader(['en'], gpu=True)  # Initialize EasyOCR

def extract_ocr_text(index):
    image_path = os.path.join(image_folder, f"{index}.jpg")
    if not os.path.exists(image_path):
        return index, ""
    
    try:
        result = reader.readtext(image_path)
        ocr_text = " ".join([text for _, text, _ in result])
    except Exception as e:
        print(f"Error processing image {index}: {e}")
        ocr_text = ""
    
    return index, ocr_text

In [8]:
# Assuming `data` is your DataFrame
data = pd.read_csv("updates.csv")
indices = data['id'].to_list()
# Process images in smaller batches to avoid memory issues
batch_size = 10  # Adjust this based on your memory constraints
ocr_results = []

for i in tqdm(range(0, len(indices), batch_size), desc="Extracting OCR"):
    batch_indices = indices[i:i + batch_size]
    batch_indices = batch_indices[::2]
    for index in batch_indices:
        batch_results = []
        batch_results.append(extract_ocr_text(index))
        ocr_results.extend(batch_results)
    

# Add the OCR results to the DataFrame
ocr_text_dict = dict(ocr_results)
data['ocr_text'] = data['id'].map(ocr_text_dict)
data = data[data['ocr_text'].notna() & (data['ocr_text'] != "")]
# Save the updated DataFrame with the OCR text to a new CSV
updated_csv_file = 'height_with_ocr.csv'
data.to_csv(updated_csv_file, index=True)

print(f"OCR data has been added to the CSV. Updated file saved as '{updated_csv_file}'.")

Extracting OCR:  18%|█▊        | 771/4360 [16:32<1:05:57,  1.10s/it]Premature end of JPEG file


Error processing image 227978: image file is truncated (4 bytes not processed)


Extracting OCR:  85%|████████▌ | 3721/4360 [1:23:35<13:09,  1.24s/it]Premature end of JPEG file


Error processing image 257474: image file is truncated (0 bytes not processed)


Extracting OCR:  86%|████████▌ | 3736/4360 [1:23:54<13:50,  1.33s/it]Premature end of JPEG file


...

Error processing image 258936: image file is truncated (0 bytes not processed)


Premature end of JPEG file
Extracting OCR:  89%|████████▊ | 3868/4360 [1:26:37<09:08,  1.11s/it]

Error processing image 258940: image file is truncated (13 bytes not processed)


Extracting OCR: 100%|██████████| 4360/4360 [1:37:18<00:00,  1.34s/it]

OCR data has been added to the CSV. Updated file saved as 'height_with_ocr.csv'.
